In [1]:
import json
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
from torch_geometric.utils import to_dense_batch
import torch.optim as optim
import json

sys.path.append(os.path.abspath(".."))
from functions.utils_L2D import load_and_restore_parquet, display_frames

/aul/homes/mtemp009/Projects/ISAC/Traffic_Semantic_Graphs/sem_graphs/lib/python3.11/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /lib64/libc.so.6: version `GLIBC_2.32' not found (required by /a/bear.cs.fiu.edu./disk/bear-b/users/mtemp009/Projects/ISAC/Traffic_Semantic_Graphs/sem_graphs/lib/python3.11/site-packages/torch_scatter/_version_cpu.so)
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


## Phase 2: Training Models without Diffusion

### A. Predicting Next Coordinates

#### 1. Load the Graphical Data

- feature_keys allows us to select which features to include in the loaded data

- include_nodes determines the node types that are included

- include_edges determines the edge types that are included

- target_spec determines which downstream task we will be training for, from the following:

    - ego_next_pos torch.Size([8, 2])
    - ego_next_heading torch.Size([8, 1])
    - ego_next_speed torch.Size([8, 1])
    - ego_next_controls torch.Size([8, 5])
    - min_dist_vehicle torch.Size([8, 1])
    - min_dist_pedestrian torch.Size([8, 1])
    - closest_vehicle_relvel_y torch.Size([8, 1])
    - ego_traj_5 torch.Size([8, 10])

- num_steps is for standardizing the episode lenght. All episodes will be cut off at this length

- If skip_short is set to true, then the episodes that are shorter than num_steps will not be loaded

In [2]:
from functions.graphs_L2D import create_dataloaders

train_loader, test_loader, train_set, test_set, norm_stats, feature_schema = create_dataloaders(
    directory="../data/graphical/L2D/",
    num_steps=5,
    batch_size=64,
    train_ratio=0.8,
    include_nodes='all',
    include_edges='all',
    feature_keys=None,
    expand_vectors=True,
    target_spec='ego_next_pos',
    skip_short=True
)

In [3]:
for key,feats in feature_schema.items():
    print(key,feats)

ego ['accel_x', 'accel_y', 'brake', 'gas', 'gear', 'heading', 'heading_error', 'latitude', 'longitude', 'prev_step_distance_m', 'speed', 'steering', 'turn_signal']
vehicle ['dist_to_ego']
pedestrian []
environment ['bicycle', 'bike_friendly', 'delta_t_prev_s', 'is_narrow', 'is_unlit', 'landuse', 'lanes', 'maxspeed', 'oneway', 'road_name', 'sidewalk', 'surface', 'traffic_controls', 'traffic_features', 'width']


In [4]:
for batch in train_loader:
    print(batch)
    print("Batch node types:", batch.node_types)
    for ntype in batch.node_types:
        print(f"{ntype} node feature shape:", batch[ntype].x.shape)
    print("Targets y shape:", getattr(batch, 'y', None).shape if hasattr(batch, 'y') else None)
    break

HeteroDataBatch(
  y=[64, 2],
  window_meta={
    episode_path=[64],
    start=[64],
    target=[64],
    frames=[64],
  },
  ego={
    x=[320, 13],
    mask=[320, 13],
    frame_id=[320],
    batch=[320],
    ptr=[65],
  },
  vehicle={
    x=[913, 1],
    mask=[913, 1],
    frame_id=[913],
    batch=[913],
    ptr=[58],
  },
  environment={
    x=[320, 15],
    mask=[320, 15],
    frame_id=[320],
    batch=[320],
    ptr=[65],
  },
  (ego, ego_to_ego, ego)={ edge_index=[2, 256] },
  (ego, ego_to_vehicle, vehicle)={ edge_index=[2, 913] },
  (environment, env_to_env, environment)={ edge_index=[2, 256] },
  (ego, ego_to_environment, environment)={ edge_index=[2, 320] }
)
Batch node types: ['ego', 'vehicle', 'environment']
ego node feature shape: torch.Size([320, 13])
vehicle node feature shape: torch.Size([913, 1])
environment node feature shape: torch.Size([320, 15])
Targets y shape: torch.Size([64, 2])


#### 2. Initialize Models

- The StudentModel class is made up of the following main components:

    - Encoders MLP: map features to hidden_dim

    - GNN: gnn_layers of HeteroConv(SAGEConv) over edge_types.

    - Readout + MLP head: select last-frame ego embedding and use MLP to predict downstream task.

In [5]:
from functions.models_L2D import StudentModel, train_one_epoch, eval_one_epoch
from torch.optim import AdamW

device = "cuda" if torch.cuda.is_available() else "cpu"
feature_dims = {ntype: len(feature_schema.get(ntype, [])) for ntype in feature_schema}
warmup_batch = next(iter(train_loader))
metadata = (warmup_batch.node_types, warmup_batch.edge_types)

model = StudentModel(
    feature_dims=feature_dims,
    metadata=metadata,
    target_spec='ego_next_pos',
    hidden_dim=128,
    gnn_layers=2,
    dropout=0.0,
).to(device)

criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

#### 3. Train Models Jointly

- In this case using MSE loss between predicted lat/lon and ground truth

In [7]:
from functions.models_L2D import compute_feature_stats, compute_target_stats

feature_norms = compute_feature_stats(train_loader, node_types=["ego","vehicle","pedestrian","environment"])
target_norm   = compute_target_stats(train_loader)

for epoch in range(10):
    tr = train_one_epoch(model, train_loader, optimizer, criterion,
                         device=device, feature_norms=feature_norms, target_norm=target_norm)
    va = eval_one_epoch(model, test_loader, criterion,
                        device=device, feature_norms=feature_norms, target_norm=target_norm)
    print(f"epoch {epoch:02d} | train {tr:.4f} | val {va:.4f}")

epoch 00 | train 0.8974 | val 0.7597
epoch 01 | train 0.6585 | val 0.5862
epoch 02 | train 0.5263 | val 0.4768
epoch 03 | train 0.4414 | val 0.4284
epoch 04 | train 0.3852 | val 0.3576
epoch 05 | train 0.3428 | val 0.3297
epoch 06 | train 0.3095 | val 0.2910
epoch 07 | train 0.2807 | val 0.2634
epoch 08 | train 0.2569 | val 0.2408
epoch 09 | train 0.2334 | val 0.2190


## Experiments

### 1. Ego Next Heading

In [8]:
target_spec = 'ego_next_heading'

train_loader, test_loader, train_set, test_set, norm_stats, feature_schema = create_dataloaders(
    directory="../data/graphical/L2D/",
    num_steps=5,
    batch_size=64,
    train_ratio=0.8,
    include_nodes='all',
    include_edges='all',
    feature_keys=None,
    expand_vectors=True,
    target_spec=target_spec,
    skip_short=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
feature_dims = {ntype: len(feature_schema.get(ntype, [])) for ntype in feature_schema}
warmup_batch = next(iter(train_loader))
metadata = (warmup_batch.node_types, warmup_batch.edge_types)

model = StudentModel(
    feature_dims=feature_dims,
    metadata=metadata,
    target_spec=target_spec,
    hidden_dim=128,
    gnn_layers=2,
    dropout=0.0,
).to(device)

criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

feature_norms = compute_feature_stats(train_loader, node_types=["ego","vehicle","pedestrian","environment"])
target_norm   = compute_target_stats(train_loader)

for epoch in range(10):
    tr = train_one_epoch(model, train_loader, optimizer, criterion,
                         device=device, feature_norms=feature_norms, target_norm=target_norm)
    va = eval_one_epoch(model, test_loader, criterion,
                        device=device, feature_norms=feature_norms, target_norm=target_norm)
    print(f"epoch {epoch:02d} | train {tr:.4f} | val {va:.4f}")

epoch 00 | train 1.5922 | val 0.5785
epoch 01 | train 0.4589 | val 0.3415
epoch 02 | train 0.3530 | val 0.3003
epoch 03 | train 0.3290 | val 0.2849
epoch 04 | train 0.3151 | val 0.2725
epoch 05 | train 0.3049 | val 0.2640
epoch 06 | train 0.2967 | val 0.2567
epoch 07 | train 0.2891 | val 0.2502
epoch 08 | train 0.2820 | val 0.2439
epoch 09 | train 0.2733 | val 0.2357


### 2. Ego Next Controls

In [10]:
target_spec = 'ego_next_controls'

train_loader, test_loader, train_set, test_set, norm_stats, feature_schema = create_dataloaders(
    directory="../data/graphical/L2D/",
    num_steps=5,
    batch_size=64,
    train_ratio=0.8,
    include_nodes='all',
    include_edges='all',
    feature_keys=None,
    expand_vectors=True,
    target_spec=target_spec,
    skip_short=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
feature_dims = {ntype: len(feature_schema.get(ntype, [])) for ntype in feature_schema}
warmup_batch = next(iter(train_loader))
metadata = (warmup_batch.node_types, warmup_batch.edge_types)

model = StudentModel(
    feature_dims=feature_dims,
    metadata=metadata,
    target_spec=target_spec,
    hidden_dim=128,
    gnn_layers=2,
    dropout=0.0,
).to(device)

criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

feature_norms = compute_feature_stats(train_loader, node_types=["ego","vehicle","pedestrian","environment"])
target_norm   = compute_target_stats(train_loader)

for epoch in range(10):
    tr = train_one_epoch(model, train_loader, optimizer, criterion,
                         device=device, feature_norms=feature_norms, target_norm=target_norm)
    va = eval_one_epoch(model, test_loader, criterion,
                        device=device, feature_norms=feature_norms, target_norm=target_norm)
    print(f"epoch {epoch:02d} | train {tr:.4f} | val {va:.4f}")

epoch 00 | train 1.0999 | val 0.4831
epoch 01 | train 0.4540 | val 0.4387
epoch 02 | train 0.4267 | val 0.4149
epoch 03 | train 0.4097 | val 0.4105
epoch 04 | train 0.3996 | val 0.3991
epoch 05 | train 0.3898 | val 0.3911
epoch 06 | train 0.3843 | val 0.3867
epoch 07 | train 0.3776 | val 0.3945
epoch 08 | train 0.3766 | val 0.3833
epoch 09 | train 0.3727 | val 0.3797


### 3. Ego Trajectory (5 steps)

In [13]:
target_spec = 'ego_traj_5'

train_loader, test_loader, train_set, test_set, norm_stats, feature_schema = create_dataloaders(
    directory="../data/graphical/L2D/",
    num_steps=5,
    batch_size=64,
    train_ratio=0.8,
    include_nodes='all',
    include_edges='all',
    feature_keys=None,
    expand_vectors=True,
    target_spec=target_spec,
    skip_short=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
feature_dims = {ntype: len(feature_schema.get(ntype, [])) for ntype in feature_schema}
warmup_batch = next(iter(train_loader))
metadata = (warmup_batch.node_types, warmup_batch.edge_types)

model = StudentModel(
    feature_dims=feature_dims,
    metadata=metadata,
    target_spec=target_spec,
    hidden_dim=128,
    gnn_layers=2,
    dropout=0.0,
).to(device)

criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

feature_norms = compute_feature_stats(train_loader, node_types=["ego","vehicle","pedestrian","environment"])
target_norm   = compute_target_stats(train_loader)

for epoch in range(10):
    tr = train_one_epoch(model, train_loader, optimizer, criterion,
                         device=device, feature_norms=feature_norms, target_norm=target_norm)
    va = eval_one_epoch(model, test_loader, criterion,
                        device=device, feature_norms=feature_norms, target_norm=target_norm)
    print(f"epoch {epoch:02d} | train {tr:.4f} | val {va:.4f}")

epoch 00 | train 71.4472 | val 1.1056
epoch 01 | train 0.7561 | val 0.5264
epoch 02 | train 0.4489 | val 0.3600
epoch 03 | train 0.3347 | val 0.3022
epoch 04 | train 0.2719 | val 0.2486
epoch 05 | train 0.2191 | val 0.1924
epoch 06 | train 0.1772 | val 0.1562
epoch 07 | train 0.1441 | val 0.1400
epoch 08 | train 0.1219 | val 0.1164
epoch 09 | train 0.1045 | val 0.1063
